In [2]:
import pandas as pd
import torch
import numpy as np 
import torchvision
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

In [3]:
import torchvision.transforms as transforms

In [4]:
# Download training dataset
dataset = MNIST(root='data/', download=True)

In [5]:
test_dataset = MNIST(root='data/', train=False)

In [6]:
dataset[0]

(<PIL.Image.Image image mode=L size=28x28 at 0x1B9BADBD9A0>, 5)

In [7]:
# exploring data
image, label = dataset[0]

In [8]:
image

In [9]:
dataset[10][0]

In [10]:
# need to convert array of images to a tensor

In [11]:
dataset = MNIST(root="data/", train=True, transform=transforms.ToTensor())

In [12]:
img_tensor, label = dataset[0]

In [13]:
print(img_tensor.shape, label)

torch.Size([1, 28, 28]) 5


In [18]:
import numpy as numpy

def split_indicies(n, val_pct):
    # size of validation set
    n_val = int(val_pct*n)

    #create random permutations of 0 to n-1
    idxs =np.random.permutation(n)

    # return first n_val indicies for validation set
    return idxs[n_val:], idxs[:n_val]



In [19]:
# 20% validation set
train_indicies, val_indicies = split_indicies(len(dataset),0.2)

In [20]:
print(len(train_indicies),len(val_indicies))

48000 12000


In [21]:
# use pytorch data loaders for each of these using a SubsetRandomSampler: randomly samples a given list of indicies and creates batches of data

In [22]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [23]:
batch_size =100

#training sampler and data loader
train_sampler = SubsetRandomSampler(train_indicies)
train_loader = DataLoader(dataset, batch_size, sampler=train_sampler)

#validation sampler and data loader
val_sampler = SubsetRandomSampler(val_indicies)
val_loader = DataLoader(dataset, batch_size, sampler=val_sampler)

In [24]:
# logistis regressiona model: x @ w.t() + b

In [25]:
import torch.nn as nn

input_size = 28*28
num_classes = 10

model = nn.Linear(input_size, num_classes)

In [26]:
model.weight.shape

torch.Size([10, 784])

In [27]:
model.bias.shape


torch.Size([10])

In [28]:
# pass our image into the initiated weights

for images, labels in train_loader:
    print(images.shape)
    outputs = model(images)
    break

torch.Size([100, 1, 28, 28])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2800x28 and 784x10)

In [29]:
# get an erro because the weights are 28*28,  but the training data is 1*28*28

In [31]:
class MinstModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, xb):
        xb = xb.reshape(-1,784)
        out = self.linear(xb)
        return out

model = MinstModel()

In [35]:
for images, labels in train_loader:
    print(images.shape)
    outputs = model(images)
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n', outputs[:2].data)

torch.Size([100, 1, 28, 28])
outputs.shape :  torch.Size([100, 10])
Sample outputs :
 tensor([[-0.2654, -0.2348, -0.0676,  0.1688, -0.1232,  0.0370, -0.0961, -0.0806,
          0.0180, -0.0199],
        [-0.3245, -0.1504, -0.0913,  0.4855, -0.0799, -0.3737, -0.1653, -0.0829,
          0.1533, -0.0501]])


In [33]:
import torch.nn.functional as F 
# includes the softmax function

In [38]:
probs = F.softmax(outputs, dim=1)
probs[:10]

tensor([[0.0813, 0.0839, 0.0991, 0.1256, 0.0938, 0.1101, 0.0964, 0.0979, 0.1080,
         0.1040],
        [0.0752, 0.0895, 0.0949, 0.1690, 0.0960, 0.0716, 0.0881, 0.0957, 0.1212,
         0.0989],
        [0.0815, 0.0893, 0.1095, 0.1082, 0.1003, 0.1006, 0.0816, 0.1163, 0.1024,
         0.1103],
        [0.0601, 0.1073, 0.0861, 0.1729, 0.0728, 0.1118, 0.0797, 0.1204, 0.1067,
         0.0821],
        [0.0838, 0.1098, 0.1134, 0.1560, 0.0672, 0.1070, 0.0895, 0.1168, 0.0794,
         0.0772],
        [0.0871, 0.1397, 0.0827, 0.1136, 0.0738, 0.0951, 0.1021, 0.0974, 0.1237,
         0.0847],
        [0.0945, 0.1286, 0.0924, 0.1036, 0.0927, 0.1121, 0.0988, 0.0961, 0.0982,
         0.0829],
        [0.0651, 0.0803, 0.0812, 0.1581, 0.0824, 0.1227, 0.0874, 0.1076, 0.1020,
         0.1132],
        [0.0715, 0.0948, 0.1129, 0.1170, 0.0894, 0.0944, 0.0734, 0.1260, 0.1024,
         0.1182],
        [0.0827, 0.0848, 0.1027, 0.1407, 0.0996, 0.0973, 0.1001, 0.1071, 0.0758,
         0.1092]], grad_fn=<

In [39]:
# get probability for each number, so need to calculate the highest probability

In [40]:
max_probs, preds = torch.max(probs, dim=1)
print(preds)

tensor([3, 3, 7, 3, 3, 1, 1, 3, 7, 3, 3, 3, 7, 3, 8, 3, 7, 7, 3, 3, 3, 8, 1, 3,
        8, 3, 1, 3, 3, 3, 3, 7, 3, 7, 7, 3, 3, 7, 7, 8, 7, 7, 3, 9, 3, 3, 7, 3,
        3, 3, 3, 7, 3, 7, 8, 3, 1, 7, 1, 3, 8, 7, 3, 3, 7, 8, 7, 9, 3, 3, 4, 8,
        3, 7, 3, 7, 3, 7, 3, 7, 7, 2, 7, 3, 3, 7, 8, 7, 1, 7, 3, 7, 3, 3, 1, 7,
        7, 7, 3, 1])


In [41]:
labels

tensor([6, 2, 1, 8, 5, 7, 7, 3, 1, 1, 1, 8, 9, 2, 2, 8, 6, 3, 9, 2, 3, 4, 7, 5,
        0, 6, 7, 6, 2, 3, 9, 8, 2, 3, 7, 8, 3, 0, 3, 2, 0, 6, 2, 7, 3, 7, 0, 3,
        4, 2, 6, 1, 3, 3, 7, 9, 9, 0, 7, 1, 4, 7, 3, 8, 3, 3, 5, 1, 6, 0, 3, 9,
        6, 0, 6, 1, 9, 3, 2, 3, 6, 3, 7, 4, 5, 5, 3, 1, 7, 9, 4, 1, 9, 2, 9, 9,
        5, 6, 2, 9])

In [42]:
def accuracy(l1, l2):
    return torch.sum(l1 == l2).item()/len(l1)

In [43]:
accuracy(preds,labels)

0.11

In [44]:
# use cross entropy as a loss function
# picks the label prediction value and natural logs it
# it is differentiable, and allows for small steps

In [45]:
loss_fn = F.cross_entropy

In [46]:
loss = loss_fn(outputs, labels)
loss

tensor(2.2974, grad_fn=<NllLossBackward>)

In [47]:
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

In [63]:
# calculates the loss for each iteration of a batch
def loss_batch(model, loss_func, xb, yb, opt=None, metric=None):

    # calculate loss
    preds = model(xb)
    loss = loss_func(preds, yb)

    if opt is not None:
        #compute gradients
        loss.backward()
        # update parameters
        opt.step()
        # reset gradients
        opt.zero_grad()
    
    metric_result = None
    if metric is not None:
        # computer the metric
        metric_result = metric(preds, yb)
    
    return loss.item(), len(xb), metric_result

In [51]:
# calculates the overall loss for the validation set

def evaluate(model, loss_fn, valid_dl, metric=None):

    # do not need to keep track of gradients
    with torch.no_grad():
        
        # pass each batch through the model
        results = [loss_batch(model,loss_fn, xb, yb, metric=metric) for xb,yb in valid_dl]

        # separate losses, counts and metrics
        losses, nums, metrics = zip(*results)

        # total size of the dataset
        total = np.sum(nums)

        # average loss across teh batches
        avg_loss = np.sum(np.multiply(losses, nums))/ total

        avg_metric = None
        if metric is not None:

            # average of metric across the batches
            avg_metric = np.sum(np.multiply(metrics, nums))/total
    
    return avg_loss, total, avg_metric

In [56]:
def accuracy(outputs,labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.sum(preds == labels).item() / len(preds)

In [59]:
evaluate(model, loss_fn, val_loader, metric=accuracy)

(2.3440155565738676, 12000, 0.07591666666666666)

In [60]:
def fit(epochs, model, loss_fn, opt, train_dl, valid_dl, metric=None):

    for epoch in range(epochs):
        
        # Training
        for xb, yb in train_dl:
            loss, _, _ = loss_batch(model,loss_fn, xb, yb, opt)

        result = evaluate(model,loss_fn, valid_dl,metric)
        val_loss, total, val_metric = result

        # Evaluating
        if metric is None:
            print("Epoch: {0}/{1}, Loss: {2}".format(epoch+1,epochs,val_loss))
        else:
            print("Epoch: {0}/{1}, Loss: {2}, {3}:{4}".format(epoch+1,epochs, val_loss,metric.__name__,val_metric))

In [66]:
# can easily reset model if needed
model = MinstModel()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate/10)

In [76]:
# training the model
fit(10, model, F.cross_entropy, optimizer, train_loader, val_loader, accuracy)

Epoch: 1/10, Loss: 1.5449748486280441, accuracy:0.74575
Epoch: 2/10, Loss: 1.5221473147471747, accuracy:0.7499166666666667
Epoch: 3/10, Loss: 1.5000946581363679, accuracy:0.7539166666666667
Epoch: 4/10, Loss: 1.4787936190764108, accuracy:0.75725
Epoch: 5/10, Loss: 1.4582076092561087, accuracy:0.7608333333333334
Epoch: 6/10, Loss: 1.4383109261592228, accuracy:0.7629166666666667
Epoch: 7/10, Loss: 1.4190746714671454, accuracy:0.7649166666666667
Epoch: 8/10, Loss: 1.4004718820254007, accuracy:0.7671666666666667
Epoch: 9/10, Loss: 1.3824775705734889, accuracy:0.7690833333333333
Epoch: 10/10, Loss: 1.3650664915641149, accuracy:0.7713333333333333


In [69]:
# testing with images

In [71]:
test_dataset = MNIST(root='data/', train=False, transform=transforms.ToTensor())

In [74]:
test_dataset[0][0][0][:5]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [75]:
def predict_image(img,model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    _, preds = torch.max(yb, dim=1)
    return preds[0].item()

In [79]:
img1, label1 = test_dataset[100]
label1, predict_image(img1,model)

(6, 6)

In [80]:
# 79% accuracy
test_loader = DataLoader(test_dataset, batch_size=200)
print("test loss, total, test accracy")
evaluate(model, loss_fn, test_loader, metric=accuracy)

test loss, total, test accracy


(1.338359444141388, 10000, 0.7901)

In [81]:
torch.save(model.state_dict(), "mnist-logistic.pth")

In [82]:
model.state_dict()

OrderedDict([('linear.weight',
              tensor([[ 0.0228, -0.0098, -0.0198,  ..., -0.0069, -0.0083, -0.0222],
                      [ 0.0202, -0.0226, -0.0082,  ..., -0.0196,  0.0058,  0.0074],
                      [-0.0344,  0.0342,  0.0307,  ..., -0.0323, -0.0241, -0.0269],
                      ...,
                      [-0.0068, -0.0096, -0.0206,  ...,  0.0143,  0.0184,  0.0094],
                      [-0.0229, -0.0052, -0.0298,  ...,  0.0143, -0.0082,  0.0273],
                      [-0.0246, -0.0205,  0.0022,  ...,  0.0022, -0.0207,  0.0028]])),
             ('linear.bias',
              tensor([-0.0332,  0.0304, -0.0080,  0.0249, -0.0105, -0.0205, -0.0211,  0.0011,
                       0.0066, -0.0232]))])

In [83]:
model2 = MinstModel()
model2.load_state_dict(torch.load("mnist-logistic.pth"))
model2.state_dict()

OrderedDict([('linear.weight',
              tensor([[ 0.0228, -0.0098, -0.0198,  ..., -0.0069, -0.0083, -0.0222],
                      [ 0.0202, -0.0226, -0.0082,  ..., -0.0196,  0.0058,  0.0074],
                      [-0.0344,  0.0342,  0.0307,  ..., -0.0323, -0.0241, -0.0269],
                      ...,
                      [-0.0068, -0.0096, -0.0206,  ...,  0.0143,  0.0184,  0.0094],
                      [-0.0229, -0.0052, -0.0298,  ...,  0.0143, -0.0082,  0.0273],
                      [-0.0246, -0.0205,  0.0022,  ...,  0.0022, -0.0207,  0.0028]])),
             ('linear.bias',
              tensor([-0.0332,  0.0304, -0.0080,  0.0249, -0.0105, -0.0205, -0.0211,  0.0011,
                       0.0066, -0.0232]))])

In [84]:
# same exact values
print("test loss, total, test accracy")
evaluate(model2, loss_fn, test_loader, metric=accuracy)

test loss, total, test accracy


(1.338359444141388, 10000, 0.7901)